In [6]:
"""
   自动钻孔机
"""
import math
import pandas as pd
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
# 数据加载
df = pd.read_csv("./drilling.csv")

# 创建距离的矩阵
# 计算两点之间的距离
size = df.shape[0] #点的个数
dist_matrix = {}  # 计算点与点之间的距离

# 欧式距离计算
def euclid_distance(x1, y1, x2, y2):
    dist = math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
    return dist

for from_node in range(size):
    dist_matrix[from_node] = {}
    for to_node in range(size):
        x1 = df.loc[from_node, 'x']
        y1 = df.loc[from_node, 'y']
        x2 = df.loc[to_node, 'x']
        y2 = df.loc[to_node, 'y']
        dist_matrix[from_node][to_node] = euclid_distance(x1, y1, x2, y2)
        
# 计算两点之间的距离
def distance_callback(from_index, to_index):
    # 从Index => NodeIndex的转换
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

In [7]:
tsp_size = df.shape[0]
num_routes = 1
depot = 0

# 创建路径规划模型Create routing model.
print(tsp_size)
print(num_routes)
print(depot)
#建路线管理，tsp_size, num_vehicles（车的数量）, depot（原点）
manager = pywrapcp.RoutingIndexManager(tsp_size, num_routes, depot)
routing = pywrapcp.RoutingModel(manager)
routing

280
1
0


<ortools.constraint_solver.pywrapcp.RoutingModel; proxy of <Swig Object of type 'operations_research::RoutingModel *' at 0x00000282732D56C0> >

In [14]:
# 设置启发式方法（距离成本最短 PATH_CHEAPEST_ARC）
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
# 在初始解的基础上，进一步优化（通过邻居调整）
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
# 搜索时间限制
search_parameters.time_limit.seconds = 5

# 创建距离回调函数，用于计算两个节点之间的距离
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# 求解路径规划
#solution = routing.SolveWithParameters(search_parameters)
search_parameters

# 求解路径规划
solution = routing.SolveWithParameters(search_parameters)

In [15]:
if solution:
    # Solution cost.
    print("Total distance: " + str(solution.ObjectiveValue()) + "\n")

    # Inspect solution.
    # Only one route here; otherwise iterate from 0 to routing.vehicles() - 1.
    route_number = 0
    node = routing.Start(route_number)
    start_node = node
    route = ''

    while not routing.IsEnd(node):
        route += str(node) + ' -> '
        node = solution.Value(routing.NextVar(node))
    route += '0'
    print("Route:\n\n" + route)
else:
    print('No solution found.')

Total distance: 2633

Route:

0 -> 1 -> 279 -> 2 -> 278 -> 277 -> 3 -> 276 -> 275 -> 274 -> 273 -> 272 -> 271 -> 270 -> 15 -> 16 -> 133 -> 269 -> 268 -> 134 -> 135 -> 267 -> 266 -> 136 -> 137 -> 138 -> 147 -> 148 -> 149 -> 178 -> 179 -> 192 -> 196 -> 197 -> 198 -> 144 -> 143 -> 142 -> 145 -> 146 -> 141 -> 140 -> 139 -> 265 -> 264 -> 263 -> 262 -> 261 -> 260 -> 259 -> 258 -> 257 -> 256 -> 255 -> 248 -> 247 -> 243 -> 242 -> 241 -> 240 -> 239 -> 238 -> 245 -> 244 -> 246 -> 249 -> 250 -> 229 -> 228 -> 231 -> 230 -> 237 -> 236 -> 235 -> 234 -> 233 -> 232 -> 227 -> 226 -> 225 -> 224 -> 223 -> 222 -> 221 -> 220 -> 219 -> 218 -> 217 -> 216 -> 215 -> 214 -> 213 -> 212 -> 211 -> 210 -> 209 -> 208 -> 251 -> 254 -> 253 -> 252 -> 207 -> 206 -> 205 -> 204 -> 203 -> 202 -> 201 -> 199 -> 200 -> 195 -> 194 -> 193 -> 191 -> 190 -> 189 -> 188 -> 187 -> 186 -> 185 -> 184 -> 183 -> 182 -> 181 -> 180 -> 175 -> 176 -> 177 -> 150 -> 151 -> 155 -> 152 -> 154 -> 153 -> 128 -> 129 -> 130 -> 131 -> 132 -> 17 -> 1